*This is an initial look into paths of classifying monet paintings. Due to current status may be worth even trying to do more genre based vs. artist based look into the situation.* 
  
*The created neural network used keras on top of a theano backend. Currently using a three stage network to categorize the images.*

In [1]:
import theano

from keras.models import Sequential        # Linear stack


from keras.layers import (                 
    # Flattened list of layers
    
                          Convolution2D,   
                            # Filters via windows of 2d input
                          MaxPooling2D,    
                            # Subsets image into matrices, largest value of matrix is taken
                          Activation,      
                            # Filters tha activate when deisred features are identified
                          Dropout,         
                            # Prevents overfitting by dropping based on probability 1-p
                          Flatten,         
                            # Layers comprising model graph
                          Dense            
                            # Specify input argument shape
    
                          )

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN not available)
Using Theano backend.


*In order to start it is important to begin by prcessing the data. This is a quick look into how it's done, will do quick demo*   
  
*Basically the image that will be looked at is taken, processed in a number of different ways (flipped, skewed, cropped, etc.). This is like turning a signature upside down for forgery. Looking at the situation through several angles.*

In [2]:
# Data Cleaning and Preparation

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



d_gen = ImageDataGenerator( featurewise_center = False, # input mean as 0 over data
                            samplewise_center = False,  # Set sample mean to 0
                            featurewise_std_normalization=False, # Standardize around std of dataset 
                            samplewise_std_normalization=False,  # Standardize around std of sample
                            zca_whitening=False, # Apply ZCA whitening
                           
                            rotation_range=0.15,   # Random Roation within specified range
                            width_shift_range=0.15, # Horizontal shift as a fractional of image width
                            height_shift_range=0.15, # Vertical shift as fraction of image height
                            shear_range=0.15, # Intensity of shear
                            zoom_range=0.15, # Either number or list, but specifies the range for the random zoom
                            
                            channel_shift_range=0., # Range for random channel shifts
                            fill_mode='nearest', # How pixels outside of the boundary are dealt with
                            cval=0., # color value for fill_mode if that value was set to 'constant'
                            horizontal_flip=True, # Random horizontal flip of sample
                            vertical_flip=True, # Random vertical flip of sample
                            rescale=1./225, # Scalar multiplying the image. 1 used  
#                             dim_ordering=K.image_dim_ordering()
                          )

In [3]:
monet1 = load_img('Claude_Monet,_Impression,_soleil_levant.jpg') # PIL image
monet_array = img_to_array(monet1) # Convert the 2d Image to an array
monet_array = monet_array.reshape((1,) + monet_array.shape) # Reshape to 1D

Show the purdy pictures.

In [4]:
t = 0
for batch in d_gen.flow(monet_array, batch_size=1,
                        save_to_dir='mod', save_prefix='monet', 
                        save_format='jpeg'):
    t += 1
    if t > 20: # Create 20 manipulated images
        break 

# Creation of the Model

*Here is where the model is created. So far the model that is being used is a rather simple three step neural network. Then going to build out a bit more*

In [5]:

model = Sequential()
model.add(Convolution2D(32,3,3, input_shape=(3,150,150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Build the model

In [6]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))           # Rectified Linear Unit
model.add(Dropout(0.5))                 # Drop 50% of the Neurons
model.add(Dense(1))
model.add(Activation('sigmoid'))        # Yay, sigmoids

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
# use the above datagen for training
d_gen = ImageDataGenerator(rescale=1./255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          horizontal_flip=True)

# For test only use a rescale function
test_d_gen = ImageDataGenerator(rescale=1./255)

In [8]:
monet_gen = d_gen.flow_from_directory('/home/sambozek/Desktop/Test/',       # Folder containing the test images
                                     target_size = (150, 150),              # Resize to 150 * 150 pxs
                                     batch_size = 32,                       # Reduce Noise, larger batch less noise
                                     class_mode='binary')                   # Binary Classification Problem

hold_out = test_d_gen.flow_from_directory('/home/sambozek/Desktop/Hold Out/',
                                         target_size= (150, 150),
                                         batch_size = 32,
                                         class_mode='binary')

Found 7295 images belonging to 2 classes.
Found 1887 images belonging to 2 classes.


In [9]:
model.fit_generator(monet_gen,                  # Resize the image get from test dictionary
                   samples_per_epoch=800,      # How many amples to use per epoch
                   nb_epoch=25,                 # Number of epochs used
                   validation_data = hold_out,  # Location of the validation data
                   nb_val_samples=1200,          # Validation samples used per epoch
                   )

model.save_weights('monet1.h5')

Epoch 1/25
800/800 [==============================] - 66s - loss: 0.3921 - acc: 0.8663 - val_loss: 2.8017 - val_acc: 0.2475
Epoch 2/25
800/800 [==============================] - 52s - loss: 0.3113 - acc: 0.8750 - val_loss: 4.0267 - val_acc: 0.2436
Epoch 3/25
800/800 [==============================] - 51s - loss: 0.3585 - acc: 0.8612 - val_loss: 3.0794 - val_acc: 0.2475
Epoch 4/25
768/800 [===========================>..] - ETA: 0s - loss: 0.2959 - acc: 0.8698

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/sambozek/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/sambozek/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/sambozek/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 404, in data_generator_task
    generator_output = next(generator)
  File "/home/sambozek/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 580, in next
    img = load_img(os.path.join(self.directory, fname), grayscale=grayscale, target_size=self.target_size)
  File "/home/sambozek/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 163, in load_img
    img = img.convert('RGB')
  File "/home/sambozek/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 842, in convert
    self.load()
  File "/home/sambozek/anaconda2/lib/python2.7/site-packages/PIL/I

800/800 [==============================] - 52s - loss: 0.2953 - acc: 0.8700 - val_loss: 5.3335 - val_acc: 0.2255
Epoch 5/25


Exception: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

## To Do

* Expand to try categorizing styles?
* Figure out a few hatches
* Try getting to above a 95% accuracy
* Manipulate loss functions
* Change the dropout rate and see what's going on
* Validation
    * ROC curve
    * Confusion Matrix
* What does model think is important?
* Restricted Boltzmann Machine for Monet Replication